In [1]:
from tkinter import *

<h2><b>GAME</b></h2>

In [4]:

class Apple :
    def __init__(self):
        pass

class Snake :
    def __init__(self):
        pass
    
class Snake :
    def __init__(self):
        pass
    
class Snake_game :
    def __init__(self):
        pass
    def play(self):
        print("Playing")
        return True

snake_game = Snake_game()
snake_game.play()

Playing


True

<h2><b>IA</b></h2>

In [6]:
class Ia_dumb :
    def __init__(self):
        pass

class Ia_smart :
    def __init__(self):
        pass
    
class Ia_q :
    def __init__(self):
        pass
    